# Imports

In [ ]:
import import_ipynb
import AlexFunctions as AF

import copy
import cv2
import matplotlib.pyplot as plt
import numpy as np
from skimage.measure import label, regionprops

# Visualize Results

In [ ]:
# BOUNDING BOX

def create_bounding_boxes(mission_numbers, anomaly_masks, output_images, num_bbox_iterations=1, bbox_colors=[(0, 0, 255), (255, 0, 0)], voting_scheme='AND', majority_success=1, display_images=False):
    combined_masks = []
    for i in range(0, len(anomaly_masks[0])): # number of differences in mission used = len(mission_numbers)-1
        if voting_scheme == 'AND':
            combined_mask = np.ones((anomaly_masks[0][0].shape[0:2])).astype(bool)
        elif voting_scheme == 'OR':
            combined_mask = np.zeros((anomaly_masks[0][0].shape[0:2])).astype(bool)
        elif voting_scheme == 'MAJORITY':
            combined_mask = np.zeros((anomaly_masks[0][0].shape[0:2])).astype(np.uint8) # number not boolean
        else:
            print('Invalid Voting Scheme Selected')

        for j in range(0, len(anomaly_masks)): # number of detection methods
            if voting_scheme == 'AND':
                combined_mask = np.logical_and(combined_mask, anomaly_masks[j][i].astype(bool))
            elif voting_scheme == 'OR':
                combined_mask = np.logical_or(combined_mask, anomaly_masks[j][i].astype(bool))
            elif voting_scheme == 'MAJORITY':
                combined_mask[anomaly_masks[j][i].astype(bool)] = combined_mask[anomaly_masks[j][i].astype(bool)] + 1

        if voting_scheme == 'MAJORITY': # compare to voting threshold
            combined_mask = combined_mask >= majority_success
        
        combined_mask = combined_mask.astype(int) # convert boolean mask to integers. True/False = 1/0
        combined_mask = (combined_mask*255).astype(np.uint8) # scale to 0-255 in order to visualize
        combined_masks.append(combined_mask)
    
    if True:
        AF.plot_images(combined_masks, ['combined_masks ' + str(mission_numbers[i]) + '-' + str(mission_numbers[i+1]) for i in range(0, len(mission_numbers)-1)])
    
    temp_bounding_box_masks = []
    num_bbox = []
    
    for k in range(0, num_bbox_iterations):
        combined_bbox_masks = []

        for i in range(0, len(combined_masks)):
            label_ = label(combined_masks[i])
            props_ = regionprops(label_)
            bounding_mask = np.zeros((combined_masks[i].shape)).astype(np.uint8)

            if k == num_bbox_iterations-1:
                num_bbox.append(len(props_))

            factor = 0
            for prop in props_:
                if k == num_bbox_iterations-1:
                    cv2.rectangle(bounding_mask, (prop.bbox[1], prop.bbox[0]), (prop.bbox[3], prop.bbox[2]), 255, 2)
                else:
                    cv2.rectangle(bounding_mask, (prop.bbox[1]-factor, prop.bbox[0]-factor), (prop.bbox[3]+2*factor, prop.bbox[2]+2*factor), 255, -1)

            combined_bbox_masks.append(bounding_mask)

        temp_bounding_box_masks.append(copy.deepcopy(combined_bbox_masks))
        temp = copy.deepcopy(combined_bbox_masks)
    
    if False:
        for k in range(0, num_bbox_iterations):
            AF.plot_images(temp_bounding_box_masks[k], ['temp_bounding_box_masks ' + str(mission_numbers[i]) + '-' + str(mission_numbers[i+1]) for i in range(0, len(mission_numbers)-1)])
    
    # ----------------------------------------
        
    colored_bbox_masks = []
    for i in range(0, len(combined_bbox_masks)):
        result = np.zeros((combined_bbox_masks[i].shape[0], combined_bbox_masks[i].shape[1], 3))
        result[combined_bbox_masks[i].astype(bool), :] = bbox_colors[0]
        colored_bbox_masks.append(result.astype(np.uint8))
        
    # display bbox 1 and 2 on image 1, display bbox 2 and 3 on image 2
    # display first set in red and second set in blue
    # 1-2, 1-2-3, 2-3-4, 4-5
    # 1    2    3    4
    superimposed_images = []
    for i in range(0, len(output_images)):
        result = copy.deepcopy(output_images[i])
        if i < len(output_images) - 1:
            result[combined_bbox_masks[i].astype(bool), :] = bbox_colors[0]
        if i > 0:
            result[combined_bbox_masks[i-1].astype(bool), :] = bbox_colors[1]
        superimposed_images.append(result.astype(np.uint8))
    
    return combined_bbox_masks, num_bbox, colored_bbox_masks, superimposed_images

In [ ]:
def compute_results(mission_numbers, heatmaps, percent_heatmap_buckets, roof_masks, percent_roof_pixels, num_bboxs, display_images=False, save_images=False):
    
    # ----------------------------------------
    
    # combine heatmap and roof mask into single image
    heatmap_roof_combined_images = []
    for i in range(0, len(mission_numbers)):
        heatmap_roof_combined_images.append(cv2.bitwise_or(heatmaps[i], roof_masks[i]).astype(np.uint8))
    
    if False:
        AF.plot_images(heatmap_roof_combined_images, ['heatmap_roof_combined_images ' + str(mission_number) for mission_number in mission_numbers])
    
    # ----------------------------------------

    metadata = [] # list of dictionaries for each mission
    for i in range(0, len(mission_numbers)):
        data = dict()
#         data['num_anomalies'] =  # TODO - from superimposed_bbox_masks
        data['percent_roof_anomalies'] = percent_roof_pixels[i]
        data['heatmap_buckets'] = np.array(percent_heatmap_buckets[i])
        if i < len(mission_numbers) - 1:
            data['num_bboxs'] = num_bboxs[i]
        metadata.append(data)
    
    # ----------------------------------------
    
    return heatmap_roof_combined_images, metadata

In [ ]:
def display_results(mission_numbers, colored_bbox_images, superimposed_bbox_images, heatmap_roof_combined_images, metadata, color_buckets):
    marker_size = 75
    # https://matplotlib.org/3.3.2/api/markers_api.html
    
    # ----------------------------------------
    
    # display bounding boxes
    AF.plot_images(colored_bbox_images, ['Anomaly Bounding Boxes: Missions ' + str(mission_numbers[i]) + ' to ' + str(mission_numbers[i+1]) for i in range(0, len(mission_numbers)-1)])
    AF.plot_images(superimposed_bbox_images, ['Superimposed Anomaly Bounding Boxes: Mission ' + str(mission_number) for mission_number in mission_numbers])
    
    # display heatmap of grass regions scores with roof anomaly mask
    AF.plot_images(heatmap_roof_combined_images, ['Grass Scored Heatmap: Mission ' + str(mission_number) for mission_number in mission_numbers])
    
    # histogram of grass region scores
    barchart_data = [metadata[i]['heatmap_buckets'] for i in range(0 , len(mission_numbers))]
    AF.plot_images(barchart_data, title_list=['Grass Scores Histogram: Mission ' + str(mission_number) for mission_number in mission_numbers], is_barchart=True, barchart_colors=color_buckets, barchart_description='SCORES', grid='on')
    
    # barchart of change in grass region scores
#     score_difference = []
#     for i in range(0, len(mission_numbers)-1):
#         diff = metadata[i+1]['heatmap_buckets'] -  metadata[i]['heatmap_buckets']
#         score_difference.append(diff)
    
#     AF.plot_images(score_difference, title_list=['Difference  Grass Scores: Mission ' + str(mission_numbers[i]) + '-' + str(mission_numbers[i+1]) for i in range(0, len(mission_numbers)-1)], is_barchart=True, barchart_colors=color_buckets, barchart_description='SCORES_DIFF', grid='on')

    # ----------------------------------------
    
    roof = 0
    boxes = 1
    score_diff = 2
    plots = (roof, boxes, score_diff)
    
    fig, axes = plt.subplots(nrows=1, ncols=len(plots), figsize=(10*len(plots), 5)) # TODO scale so that it fills entire output box
    
    # ----------------------------------------
    
    # scatter plot of percent anomaly pixels within roof image
    percent_roof_anomalies = [metadata[i]['percent_roof_anomalies'] for i in range(0, len(mission_numbers))]
    
    axes[roof].scatter(mission_numbers, percent_roof_anomalies, c='k', marker='o', s=marker_size)
    axes[roof].set_title('Anomaly Roof Percentage for Each Mission')
    axes[roof].set_xlabel('Mission Number')
    axes[roof].set_ylabel('Percentage of Roof Pixels Classified as Anomalies (%)')
    axes[roof].set_xticks(mission_numbers)
    axes[roof].set_ylim([0, np.round(np.max(percent_roof_anomalies)*1.5)])
    
    # ----------------------------------------

    # scatter plot of number of bounding boxes
    num_bboxs = [metadata[i]['num_bboxs'] for i in range(0, len(mission_numbers)-1)]
    total_num_bboxs = copy.deepcopy(num_bboxs)
    total_num_bboxs.append(0)
    total_num_bboxs = np.array(total_num_bboxs)
    total_num_bboxs[1:] += total_num_bboxs[0:len(total_num_bboxs)-1]
        
    axes[boxes].scatter(mission_numbers[0:len(mission_numbers)-1], num_bboxs, c='r', marker='<', s=marker_size)
    axes[boxes].scatter(mission_numbers[1:], num_bboxs, c='b', marker='>', s=marker_size)
    axes[boxes].plot(mission_numbers, total_num_bboxs, c='k')
    axes[boxes].set_title('Number of Anomalies Detected for each Mission')
    axes[boxes].set_xlabel('Mission Number')
    axes[boxes].set_ylabel('Number of Anomalies Detected')
    axes[boxes].set_xticks(mission_numbers)
#     axes[boxes].set_yticks(np.arange(0, np.round(np.max(num_bboxs)*1.5+1)))
    axes[boxes].set_ylim([0, np.round(np.max(num_bboxs)*1.5+1)])
    
    # ----------------------------------------
    
    # plot of changes in grass scores
    score_differences = [] 
    for i in range(0, len(mission_numbers)-1):
        diff = metadata[i+1]['heatmap_buckets'] - metadata[i]['heatmap_buckets']
        score_differences.append(diff)
    
    for i in range(0, len(score_differences[0])):
        axes[score_diff].plot(np.arange(0, len(score_differences)), [s[i] for s in score_differences], c=np.array(color_buckets[i][::-1]).astype(float)/256)
    
    axes[score_diff].set_title('Change in Grass Score')
    axes[score_diff].set_xlabel('Adjacent Missions')
    axes[score_diff].set_ylabel('Percent Change in Adjacent Mission Scores (%)')
    axes[score_diff].set_xticks(np.arange(0, len(mission_numbers)-1))
    axes[score_diff].set_xticklabels([str(mission_numbers[i]) + 'to' + str(mission_numbers[i+1]) for i in range(0, len(mission_numbers)-1)], rotation='horizontal', fontsize=10)
#     axes[score_diff].set_yticks(np.arange(np.round(np.min(np.min(score_differences)))*1.1, np.round(np.max(np.max(score_differences)))*1.1))
#     axes[score_diff].set_ylim([np.round(np.min(np.min(score_differences)))*1.1, np.round(np.max(np.max(score_differences)))*1.1])

In [ ]:
# display percent change in heatmap scores
def display_heatmap_change_scores(mission_numbers, heatmap_percents, color_buckets, scores=[0]):
    if np.min(scores) < 0:
        print('[Error] Invalid Score:', np.min(scores))
    elif np.max(scores) > len(color_buckets)-1:
        print('[Error] Invalid Score:', np.max(scores))
    else:
        score_differences = []
        for i in range(0, len(heatmap_percents)-1):
            diff = heatmap_percents[i+1] - heatmap_percents[i]
            score_differences.append(diff)

        fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

        print('Displaying Scores:', scores)
        for i in scores:
            axes.plot(np.arange(0, len(score_differences)), [s[i] for s in score_differences], c=np.array(color_buckets[i][::-1]).astype(float)/256)

        axes.set_title('Change in Grass Score')
        axes.set_xlabel('Adjacent Missions')
        axes.set_ylabel('Percent Change in Adjacent Mission Scores (%)')
        axes.set_xticks(np.arange(0, len(mission_numbers)-1))
        axes.set_xticklabels([str(mission_numbers[i]) + 'to' + str(mission_numbers[i+1]) for i in range(0, len(mission_numbers)-1)], rotation='horizontal', fontsize=10)

# Perform Analysis

In [ ]:
# def perform_analysis(mission_numbers, metadata):
#     print('Metadata')
#     for i in range(0, len(mission_numbers)):
#         print(mission_numbers[i])
#         print(metadata[i])
        
    # greatest change in score mission to mission
    # no change greater than 

# Summary

In [ ]:
def run_visualize_results(mission_numbers, anomaly_masks, histogram_mission_images_bgr, num_bbox_iterations, bbox_colors, heatmaps, percent_heatmap_buckets, roof_masks, percent_roof_pixels, color_buckets, voting_scheme, majority_success, display_images=False):
    print('Visualize Results')
    print('Step 1 - merging anomaly masks')
    combined_bbox_masks, num_bboxs, colored_bbox_images, superimposed_bbox_images = create_bounding_boxes(mission_numbers, anomaly_masks, histogram_mission_images_bgr, num_bbox_iterations, bbox_colors, voting_scheme, majority_success, display_images)
    print('Step 2 - computing results')
    heatmap_roof_combinations, metadata = compute_results(mission_numbers, heatmaps, percent_heatmap_buckets, roof_masks, percent_roof_pixels, num_bboxs, display_images)
    print('Step 3 - displaying results')
    display_results(mission_numbers, colored_bbox_images, superimposed_bbox_images, heatmap_roof_combinations, metadata, color_buckets)
#     print('Step 4 - performing analysis')
#     perform_analysis(mission_numbers, metadata)
    print('Done')